<a href="https://colab.research.google.com/github/shubham2279/FL-CATE/blob/main/FL_CAT_Single_Sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ###############   Base FL    ###################
# from datetime import datetime
# import random as ran
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
# from tensorflow.keras.layers import Dense, Flatten, Conv2D
# from tensorflow.keras import Model
# print("Tensorflow version = ", tf. __version__)
# # recipe is used to calculate object size in bytes and suggested to be used by
# # python documentation
# # import sys as re
# # pympler is used to calculate object size in bytes
# !pip install pympler
# from pympler import asizeof # importing the asizeof function specifically
# import sys

# # *****************************************************************************

# # constant variables
# # learning rate
# ALPHA = 0.1
# FLUX = 0.15 # percentage value 0-1
# ENERGY = 4 # energy units cost per bit
# WEIGHTS = 15 # max weight
# NODES = 6 # num of nodes in network including sink
# MAX_COST = 1000
# TRANSMITION_COST = 10
# ITERATIONS = 5
# # *****************************************************************************
# # setup of the neural network model's layers initialisation
# class Model_Node(Model):
#     # initialise model using the tensorflow recommended layers

#     grads = []
#     num_of_vars = 0
#     node_id = None

#     def __init__(self, nid):
#         super(Model_Node, self).__init__()
#         self.conv1 = Conv2D(32, 3, activation='relu')
#         self.flatten = Flatten()
#         self.d1 = Dense(128, activation='relu')
#         self.d2 = Dense(10)
#         self.num_of_vars = -1
#         self.node_id = nid
#         # Initialize the optimizer here to ensure it's aware of the model's variables
#         self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

#     def call(self, x):
#         x = self.conv1(x)
#         x = self.flatten(x)
#         x = self.d1(x)
#         return self.d2(x)
#     def __getitem__(self, item): return self
#     # initialising loss computatuin function
#     loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
#         from_logits=True)

#     # # initialising stochastic gradient descent function
#     # optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

#     # setting up the metrics collection tensors
#     train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
#     train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
#         name='train_accuracy'+str(node_id))

#     test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
#     test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
#         name='test_accuracy'+str(node_id))

#     # model train function
#     @tf.function
#     def train_step(self, images, labels, glo=False, g=None):
#         # if glo=true then it updates the global model using the gradient
#         # given in argument g
#         if glo:
#             # self.optimizer.apply_gradients(zip(g, self.trainable_variables),experimental_aggregate_gradients=True)
#             self.optimizer.apply_gradients(zip(g, self.trainable_variables))
#             return
#         with tf.GradientTape() as tape:
#             predictions = self(images, training=True)
#             loss = self.loss_object(labels, predictions)
#         gradients = tape.gradient(loss, self.trainable_variables);
#         if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
#         # applies training gradients using stochastic gradient descend
#         # self.optimizer.apply_gradients(zip(gradients,self.trainable_variables),experimental_aggregate_gradients=False)
#         self.optimizer.apply_gradients(zip(gradients,
#                                            self.trainable_variables))
#         # calculates loss and accuracy
#         self.train_loss(loss)
#         self.train_accuracy(labels, predictions)
#         x = self.num_of_vars
#         # x=self.num
#         self.grads = gradients[-x:]
#         # returns the gradient
#         return self.grads

#     # model predict function
#     @tf.function
#     def test_step(self, images, labels):
#         # predicts the label
#         predictions = self(images, training=False)
#         t_loss = self.loss_object(labels, predictions)
#         # calculates loss and accuracy
#         self.test_loss(t_loss)
#         self.test_accuracy(labels, predictions)

# # *****************************************************************************

# # function for averaging the gradients
# def avgs(matrices = []):
#     segmentIDs = [0] * len(matrices)
#     return tf.math.segment_mean(matrices, segmentIDs)

# # function returning the shortest paths for each node
# def shortest_paths():
#     paths = []
#     for i in range(1, NODES):
#         path_cost = MAX_COST
#         path = None
#         for p in fwd_table[i]:
#             cost = len(p) * TRANSMITION_COST
#             for hop in p: cost += edges[hop]
#             if cost < path_cost: path_cost, path = cost, p
#         paths.append((path, path_cost))
#     return paths

# # *****************************************************************************

# # array of nodes in network
# nodes = [0, 1, 2, 3, 4, 5]
# # dictionary of edges in the graph and their weights
# edges = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0,
#          'f': 0, 'g': 0, 'h': 0, 'i': 0}
# # forwarding table containing all routing paths
# fwd_table = [None, ['a', 'cb'], ['b', 'ca'],
#              ['da', 'dcb', 'iea', 'iecb', 'ifb', 'ifca'],
#              ['ea', 'ecb', 'fb', 'fca'],
#              ['gb', 'gca', 'hfb', 'hfca', 'hea', 'hecb']]

# # *****************************************************************************

# start_time =  datetime.now().strftime("%H:%M:%S")

# # sim run
# mnist = tf.keras.datasets.mnist
# print("loading dataset...")
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0
# x_train = x_train[..., tf.newaxis].astype("float32")
# x_test = x_test[..., tf.newaxis].astype("float32")
# train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(5)
# test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# init = None
# nds = [[], [], [], [], []]
# i = 0
# for img, lbl in train_ds:
#         if i == 0: init = (img, lbl); i += 1; continue
#         nds[i%5].append((img, lbl))
#         i += 1
# print("dataset split")
# # *****************************************************************************
# print("initialiasing nodes")
# nodes = []
# for i in range(0, NODES):
#     nodes.append(Model_Node(i)); print("node", i, "initialised")
#     nodes[-1].train_loss.reset_state()
#     nodes[-1].train_accuracy.reset_state()
#     nodes[-1].test_loss.reset_state()
#     nodes[-1].test_accuracy.reset_state()
# # *****************************************************************************
# j = 0
# grad, acc, bits = [], [], []
# gs = [[], [], [], [], [], []]
# # initial train of all nodes
# for i in range(0, NODES):
#     nodes[i].train_step(init[0], init[1])
#     nodes[i].set_weights(nodes[0].get_weights())
# # main training and testing loop
# for i in range(0, len(nds[0])):
#     print("Iteration: ", i)
#     # train the local models
#     for j in range(1, NODES):
#         print("training local model ", j)
#         g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
#         # save gradients in list
#         for k in range(0, 6): gs[k].append(g[k])
#     print("computing avgs")
#     for j in range(0, 6): grad.append(avgs(gs[j])[0])
#     print("avgs computed")
#     # update the global model
#     nodes[0].train_step(None, None, glo=True, g=grad)
#     print("global model updated")
#     # update local models on the updated global
#     for j in range(1, NODES): nodes[j].set_weights(nodes[0].get_weights())
#     # test updated global model
#     for ti, tl in test_ds: nodes[0].test_step(ti, tl)
#     # gather metrics for graphs
#     acc.append(nodes[0].test_accuracy.result() * 100)
#     print("Accuracy: ", acc[-1])
#     size = 0
#     # for g in grad: size += re.total_size(g)
#     for g in grad: size += asizeof.asizeof(g)
#     bits.append(size)
#     print("Bits: ", size)
#     # reset variables for next iteration
#     grad = []
#     gs = [[], [], [], [], [], []]
#     nodes[0].test_loss.reset_state()
#     nodes[0].test_accuracy.reset_state()
# # run another last iteration to account for the leftover records
# for i in range(2, NODES):
#     g = nodes[i].train_step(nds[i-1][-1][0], nds[i-1][-1][1])
#     for k in range(0, 6): gs[k].append(g[k])
# for j in range(0, 6): grad.append(avgs(gs[j])[0])
# # final update of the global model
# nodes[0].train_step(None, None, glo=True, g=grad)
# # test updated global model
# for ti, tl in test_ds: nodes[0].test_step(ti, tl)
# # gather metrics for graphs
# acc.append(nodes[0].test_accuracy.result() * 100)
# print("Accuracy: ", acc[-1])
# size = 0
# # for g in grad: size += re.total_size(g)
# for g in grad: size += asizeof.asizeof(g)
# bits.append(size)
# print("Bits: ", size)
# # *****************************************************************************
# # plot and save graphs to file
# print("plotting figures")
# plt.figure(1, figsize=(20,20))
# plt.title('FL Accuracy', fontsize=25)
# plt.plot(range(0, len(acc)), acc, alpha=0.8)
# plt.savefig("baseline_acc.png", bbox_inches='tight')

# plt.figure(2, figsize=(20,20))
# plt.title('FL size', fontsize=25)
# plt.plot(range(0, len(bits)), bits, alpha=0.8)
# plt.savefig("baseline_size.png", bbox_inches='tight')


# print("script finished running")
# print("start time: ", start_time, " --------- end time:",
#       datetime.now().strftime("%H:%M:%S"))


Tensorflow version =  2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 6.2 MB/s eta 0:00:00
loading dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
dataset split
initialiasing nodes
node 0 initialised
node 1 initialised
node 2 initialised
node 3 initialised
node 4 initialised
node 5 initialised


Iteration:  0
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
computing avgs
avgs computed
global model updated
Accuracy:  tf.Tensor(14.16, shape=(), dtype=float32)
Bits:  1392
Iteration:  1
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
computing avgs
avgs computed
global model updated
Accuracy:  tf.Tensor(23.74, shape=(), dtype=float32)
Bits:  1392
Iteration:  2
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
computing avgs
avgs computed
global model updated
Accuracy:  tf.Tensor(22.390001, shape=(), dtype=float32)
Bits:  1392
Iteration:  3
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
computing avgs
avgs computed
global model updated
Accuracy:  tf.Tensor(35.85, shape=(), dtype=float32)
Bits:  1392


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-1ea2648507be>", line 199, in <cell line: 182>
    for ti, tl in test_ds: nodes[0].test_step(ti, tl)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 826, in __next__
    return self._next_internal()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 776, in _next_internal
    ret = gen_dataset_ops.iterator_get_next(
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 3081, in iterator_get_next
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiv

TypeError: object of type 'NoneType' has no len()

In [6]:
##################    Base FL (Federated Averaging)   ##################
from datetime import datetime
import random as ran
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import sys

# *****************************************************************************

# constant variables
# learning rate
ALPHA = 0.1
FLUX = 0.15 # percentage value 0-1
ENERGY = 2 # energy units cost per bit
WEIGHTS = 15 # max weight
NODES = 6 # num of nodes in network including sink
MAX_COST = 1000
TRANSMITION_COST = 10
SGD_STEPS = 5
# *****************************************************************************
# array of nodes in network
paths = {3 : "310", 4: "420", 5: "520", 1 : "10", 2 : "20"}
costs = {0: 0, 1: 3, 2: 1, 3: 0.5, 4: 2 , 5:1.5}
# *****************************************************************************
# setup of the neural network model's layers initialisation
class Model_Node(Model):
    # initialise model using the tensorflow recommended layers

    grads = []
    num_of_vars = 0
    node_id = None

    def __init__(self, nid):
        super(Model_Node, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        self.num_of_vars = -1
        self.node_id = nid
        # Initialize the optimizer here to ensure it's aware of the model's variables
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    def __getitem__(self, item): return self
    # initialising loss computatuin function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)

    # initialising stochastic gradient descent function
    optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    # setting up the metrics collection tensors
    train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy'+str(node_id))

    test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='test_accuracy'+str(node_id))

    # model train function
    @tf.function
    def train_step(self, images, labels, glo=False, g=None):
        # if glo=true then it updates the global model using the gradient
        # given in argument g
        if glo:
            # self.optimizer.apply_gradients(zip(g, self.trainable_variables), experimental_aggregate_gradients=True)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            return
        with tf.GradientTape() as tape:
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables);
        if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
        # applies training gradients using stochastic gradient descend
        # self.optimizer.apply_gradients(zip(gradients, self.trainable_variables), experimental_aggregate_gradients=False)
        self.optimizer.apply_gradients(zip(gradients,
                                           self.trainable_variables))
        # calculates loss and accuracy
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)
        x = self.num_of_vars
        self.grads = gradients[-x:]
        # returns the gradient
        return self.grads

    # model predict function
    @tf.function
    def test_step(self, images, labels):
        # predicts the label
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        # calculates loss and accuracy
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

# *****************************************************************************
# function for averaging the gradients
def avgs(matrices = []):
    segmentIDs = [0] * len(matrices)
    return tf.math.segment_mean(matrices, segmentIDs)

# function returning the enegry cost
def compute_transmission_cost(node):
    cost = 0
    for c in paths[node]: cost += (costs[int(c)] * ENERGY)
    return cost
# *****************************************************************************
start_time =  datetime.now().strftime("%H:%M:%S")

# sim run
mnist = tf.keras.datasets.mnist
print("loading dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(5)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

init = None
nds = [[], [], [], [], []]
i = 0
for img, lbl in train_ds:
        if i == 0: init = (img, lbl); i += 1; continue
        nds[i%5].append((img, lbl))
        i += 1
print("dataset split")
# *****************************************************************************
print("initialiasing nodes")
nodes = []
for i in range(0, NODES):
    nodes.append(Model_Node(i)); print("node", i, "initialised")
    nodes[-1].train_loss.reset_state()
    nodes[-1].train_accuracy.reset_state()
    nodes[-1].test_loss.reset_state()
    nodes[-1].test_accuracy.reset_state()
# *****************************************************************************
j = 0
ecost = 0
ecosts, acc, los, bits = [], [], [], []
gs = [[], [], [], [], [], []]
# initial train of all nodes
for i in range(0, NODES):
    nodes[i].train_step(init[0], init[1])
    if i == 0: continue
    nodes[i].set_weights(nodes[0].get_weights())
# main training and testing loop
for i in range(0, len(nds[0])):
    idiv = i % SGD_STEPS
    print("Iteration: ", i)
    # train the local models
    for j in range(1, NODES):
        print("training local model ", j)
        g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
        # save gradients in list
        #for k in range(0, 6): gs[k].append(g[k])
    if idiv == (SGD_STEPS - 1):
        ws = nodes[0].get_weights()
        for j in range(1, NODES):
            g = nodes[j].get_weights()
            for k in range(0, 6): gs[k].append(g[k])
        for j in range(0, 6):
            for k in range(0, 5): gs[j][k] = ws[j] - gs[j][k]
        for j in range(0, 6): ws[j] = (gs[j][0] + gs[j][1] + gs[j][2]
                                       + gs[j][3] + gs[j][4]) / 5
        # nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables), experimental_aggregate_gradients=True)
        nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))
        # gather metrics for graphs
        for ti, tl in test_ds: nodes[0].test_step(ti, tl)
        acc.append(nodes[0].test_accuracy.result() * 100)
        los.append(nodes[0].test_loss.result())
        print("Accuracy: ", acc[-1])
        print("Loss: ", los[-1])
        for j in range(1, 6):
            ct = compute_transmission_cost(j)
            for k in range(0, 6): ecost += (gs[k][j-1].nbytes * 8 * ct)
        ecosts.append(ecost)
        print("Energy cost: ", ecost)
        size = 0
        for g in gs:
            for w in g: size += w.nbytes * 8
        bits.append(size)
        print("Bits: ", bits[-1])
        gs = [[], [], [], [], [], []]
        for j in range(1, NODES):
            nodes[j].set_weights(nodes[0].get_weights())
        nodes[0].test_loss.reset_state()
        nodes[0].test_accuracy.reset_state()
# run another last iteration to account for the leftover records
print("Iteration: ", i+1)
print("Remainder: ", i % SGD_STEPS)
ws = nodes[0].get_weights()
for j in range(1, NODES):
    g = nodes[j].get_weights()
    for k in range(0, 6): gs[k].append(g[k])
for j in range(0, 6):
    for k in range(0, 5): gs[j][k] = ws[j] - gs[j][k]
for j in range(0, 6): ws[j] = (gs[j][0] + gs[j][1] + gs[j][2]
                               + gs[j][3] + gs[j][4]) / 5
nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
                                   experimental_aggregate_gradients=True)
# gather metrics for graphs
for ti, tl in test_ds: nodes[0].test_step(ti, tl)
acc.append(nodes[0].test_accuracy.result() * 100)
los.append(nodes[0].test_loss.result())
print("Accuracy: ", acc[-1])
print("Loss: ", los[-1])
for j in range(1, 6):
    ct = compute_transmission_cost(j)
    for k in range(0, 6): ecost += (gs[k][j-1].nbytes * 8 * ct)
ecosts.append(ecost)
print("Energy cost: ", ecost)
size = 0
for g in gs:
    for w in g: size += w.nbytes * 8
bits.append(size)
print("Bits: ", bits[-1])
# *****************************************************************************
# plot and save graphs to file
print("plotting figures")
plt.figure(1, figsize=(20,20))
plt.title('FL Accuracy', fontsize=25)
plt.plot(range(0, len(acc)), acc, alpha=0.8)
pngname = "baseline_acc_%d" % SGD_STEPS
pngname += "sgd.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(2, figsize=(20,20))
plt.title('FL size', fontsize=25)
plt.plot(range(0, len(bits)), bits, alpha=0.8)
pngname = "baseline_size_%d" % SGD_STEPS
pngname += "sgd.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(3, figsize=(20,20))
plt.title('FL Energy Consupmtioon', fontsize=25)
plt.plot(range(0, len(ecosts)), ecosts, alpha=0.8)
pngname = "baseline_energy_%d" % SGD_STEPS
pngname += "sgd.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')
# *****************************************************************************
print("script finished running")
print("start time: ", start_time, " --------- end time:",
      datetime.now().strftime("%H:%M:%S"))

Streaming output truncated to the last 5000 lines.
training local model  4
training local model  5
Accuracy:  tf.Tensor(89.54, shape=(), dtype=float32)
Loss:  tf.Tensor(0.42939928, shape=(), dtype=float32)
Energy cost:  368826798080.0
Bits:  443301440
Iteration:  800
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  801
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  802
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  803
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  804
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Accuracy:  tf.Tensor(89.61, shape=(), dtype=float32)
Loss:  tf.Tenso

KeyboardInterrupt: 

In [7]:
################    Threshold Based Sparsification    ############################
from datetime import datetime
import random as ran
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import sys

# *****************************************************************************

# constant variables
ALPHA = 0.1 # learning rate
ENERGY = 2 # energy units cost per bit
NODES = 6 # num of nodes in network including sink
SGD_STEPS = 5 # num of stochastic gradient descent steps in between updates
SPARSIFICATION_THRESHOLD = 0.001
# *****************************************************************************
# array of nodes in network
paths = {3 : "310", 4: "420", 5: "520", 1 : "10", 2 : "20"}
costs = {0: 0, 1: 3, 2: 1, 3: 0.5, 4: 2 , 5:1.5}
# *****************************************************************************
# setup of the neural network model's layers initialisation
class Model_Node(Model):
    # initialise model using the tensorflow recommended layers

    grads = []
    num_of_vars = 0
    node_id = None

    def __init__(self, nid):
        super(Model_Node, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        self.num_of_vars = -1
        self.node_id = nid
        # Initialize the optimizer here to ensure it's aware of the model's variables
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    def __getitem__(self, item): return self
    # initialising loss computatuin function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)

    # initialising stochastic gradient descent function
    # optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    # setting up the metrics collection tensors
    train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy'+str(node_id))

    test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='test_accuracy'+str(node_id))

    # model train function
    @tf.function
    def train_step(self, images, labels, glo=False, g=None):
        # if glo=true then it updates the global model using the gradient
        # given in argument g
        if glo:
            # self.optimizer.apply_gradients(zip(g, self.trainable_variables), experimental_aggregate_gradients=True)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            return
        with tf.GradientTape() as tape:
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables);
        if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
        # applies training gradients using stochastic gradient descend
        # self.optimizer.apply_gradients(zip(gradients, self.trainable_variables), experimental_aggregate_gradients=False)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # calculates loss and accuracy
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)
        x = self.num_of_vars
        self.grads = gradients[-x:]
        # returns the gradient
        return self.grads

    # model predict function
    @tf.function
    def test_step(self, images, labels):
        # predicts the label
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        # calculates loss and accuracy
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

# *****************************************************************************
# function for averaging the gradients
def avgs(matrices = []):
    segmentIDs = [0] * len(matrices)
    return tf.math.segment_mean(matrices, segmentIDs)

# function returning the enegry cost
def compute_transmission_cost(node):
    cost = 0
    for c in paths[node]: cost += (costs[int(c)] * ENERGY)
    return cost
# *****************************************************************************
start_time =  datetime.now().strftime("%H:%M:%S")

# sim run
mnist = tf.keras.datasets.mnist
print("loading dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(5)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

init = None
nds = [[], [], [], [], []]
i = 0
for img, lbl in train_ds:
        if i == 0: init = (img, lbl); i += 1; continue
        nds[i%5].append((img, lbl))
        i += 1
print("dataset split")
# *****************************************************************************
print("initialiasing nodes")
nodes = []
for i in range(0, NODES):
    nodes.append(Model_Node(i)); print("node", i, "initialised")
    nodes[-1].train_loss.reset_state()
    nodes[-1].train_accuracy.reset_state()
    nodes[-1].test_loss.reset_state()
    nodes[-1].test_accuracy.reset_state()
# *****************************************************************************
j, ecost = 0, 0
ecosts, acc, los, bits = [], [], [], []
gs = [[], [], [], [], [], []]
# initial train of all nodes
for i in range(0, NODES):
    nodes[i].train_step(init[0], init[1])
    if i == 0: continue
    nodes[i].set_weights(nodes[0].get_weights())
# main training and testing loop
for i in range(0, len(nds[0])):
    print("Iteration: ", i)
    # train the local models
    for j in range(1, NODES):
        print("training local model ", j)
        g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
    idiv = i % SGD_STEPS # checks how many steps of SGD have been run
    if idiv == (SGD_STEPS - 1):
        ws = nodes[0].get_weights()
        nd = [0] * (NODES - 1)
        # sparsify gradient (client side)
        for j in range(1, NODES):
            g = nodes[j].get_weights()
            totsize, size = 0, 0
            for k in range(0, 6): g[k] = ws[k] - g[k]
            for gg, k in zip(g, range(0, 6)):
                totsize += gg.nbytes
                gg[np.abs(gg) < SPARSIFICATION_THRESHOLD] = 0
                ind = gg.nonzero()
                compressed = gg[ind]
                size += compressed.nbytes + sys.getsizeof(ind)
                gs[k].append((ind, compressed))
            percentage = (size / totsize) * 100
            nd[j-1] = size
            print("Node ", j, ": sparsified and sent ",
                  percentage, "% of original gradient")
        # average differences (server side)
        for j in range(0, 6):
            l = len(gs[j])
            if l == 0: ws[j] = np.zeros(ws[j].shape, dtype="float32"); continue
            s = np.zeros(ws[j].shape, dtype="float32")
            for g in gs[j]:
                decompressed = np.zeros(ws[j].shape, dtype="float32")
                decompressed[g[0]] = g[1]
                s += decompressed
            ws[j] = s / l
        # update global model
        # nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables), experimental_aggregate_gradients=True)
        nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))
        # gather metrics for graphs
        for ti, tl in test_ds: nodes[0].test_step(ti, tl)
        acc.append(nodes[0].test_accuracy.result() * 100)
        los.append(nodes[0].test_loss.result())
        print("Accuracy: ", acc[-1])
        print("Loss: ", los[-1])
        for j in range(1, 6):
            ct = compute_transmission_cost(j)
            ecost += ct * nd[j-1] * 8
        ecosts.append(ecost)
        print("Energy cost: ", ecost)
        size = 0
        for n in nd: size += n * 8
        bits.append(size)
        print("Bits: ", bits[-1])
        gs = [[], [], [], [], [], []]
        for j in range(1, NODES):
            nodes[j].set_weights(nodes[0].get_weights())
        nodes[0].test_loss.reset_state()
        nodes[0].test_accuracy.reset_state()
# run another last iteration to account for the leftover records
print("Iteration: ", i+1)
print("Remainder: ", i % SGD_STEPS)
ws = nodes[0].get_weights()
nd = [0] * (NODES - 1)
# sparsify gradient
# sparsify gradient (client side)
for j in range(1, NODES):
    g = nodes[j].get_weights()
    totsize, size = 0, 0
    for k in range(0, 6): g[k] = ws[k] - g[k]
    for gg, k in zip(g, range(0, 6)):
        totsize += gg.nbytes
        gg[np.abs(gg) < SPARSIFICATION_THRESHOLD] = 0
        ind = gg.nonzero()
        compressed = gg[ind]
        size += compressed.nbytes + sys.getsizeof(ind)
        gs[k].append((ind, compressed))
    percentage = (size / totsize) * 100
    nd[j-1] = size
    print("Node ", j, ": sparsified and sent ",
          percentage, "% of original gradient")
# average differences (server side)
for j in range(0, 6):
    l = len(gs[j])
    if l == 0: ws[j] = np.zeros(ws[j].shape, dtype="float32"); continue
    s = np.zeros(ws[j].shape, dtype="float32")
    for g in gs[j]:
        decompressed = np.zeros(ws[j].shape, dtype="float32")
        decompressed[g[0]] = g[1]
        s += decompressed
    ws[j] = s / l
# update global model
# nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables), experimental_aggregate_gradients=True)
nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))

# gather metrics for graphs
for ti, tl in test_ds: nodes[0].test_step(ti, tl)
acc.append(nodes[0].test_accuracy.result() * 100)
los.append(nodes[0].test_loss.result())
print("Accuracy: ", acc[-1])
print("Loss: ", los[-1])
for j in range(1, 6):
    ct = compute_transmission_cost(j)
    ecost += ct * nd[j-1] * 8
ecosts.append(ecost)
print("Energy cost: ", ecost)
size = 0
for n in nd: size += n * 8
bits.append(size)
print("Bits: ", bits[-1])
# *****************************************************************************
# plot and save graphs to file
print("plotting figures")
plt.figure(1, figsize=(20,20))
plt.title('FL Accuracy', fontsize=25)
plt.plot(range(0, len(acc)), acc, alpha=0.8)
pngname = "baseline_acc_sparsified.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(2, figsize=(20,20))
plt.title('FL size', fontsize=25)
plt.plot(range(0, len(bits)), bits, alpha=0.8)
pngname = "baseline_size_sparsified.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(3, figsize=(20,20))
plt.title('FL Energy Consupmtioon', fontsize=25)
plt.plot(range(0, len(ecosts)), ecosts, alpha=0.8)
pngname = "baseline_energy_sparsified.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')
# *****************************************************************************
print("script finished running")
print("start time: ", start_time, " --------- end time:",
      datetime.now().strftime("%H:%M:%S"))

loading dataset...
dataset split
initialiasing nodes
node 0 initialised
node 1 initialised
node 2 initialised
node 3 initialised
node 4 initialised
node 5 initialised


Streaming output truncated to the last 5000 lines.
training local model  3
training local model  4
training local model  5
Node  1 : sparsified and sent  5.292037851264367 % of original gradient
Node  2 : sparsified and sent  5.244900625632978 % of original gradient
Node  3 : sparsified and sent  4.186298154140893 % of original gradient
Node  4 : sparsified and sent  3.873373386741085 % of original gradient
Node  5 : sparsified and sent  5.955604385133511 % of original gradient
Accuracy:  tf.Tensor(85.39, shape=(), dtype=float32)
Loss:  tf.Tensor(0.66961974, shape=(), dtype=float32)
Energy cost:  9841847456.0
Bits:  21768064
Iteration:  465
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  466
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  467
training local model  1
training local model  2
training local model  3
trainin

KeyboardInterrupt: 

In [8]:
############  CONSTANT T SPARSIFICATION   #############
from datetime import datetime
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import sys

# *****************************************************************************

# constant variables
ALPHA = 0.1 # learning rate
ENERGY = 2 # energy units cost per bit
NODES = 6 # num of nodes in network including sink
SGD_STEPS = 5 # num of stochastic gradient descent steps in between updates
T_PERCENTAGE = 0.04
# *****************************************************************************
# array of nodes in network
paths = {3 : "310", 4: "420", 5: "520", 1 : "10", 2 : "20"}
costs = {0: 0, 1: 3, 2: 1, 3: 0.5, 4: 2 , 5:1.5}
# *****************************************************************************
# setup of the neural network model's layers initialisation
class Model_Node(Model):
    # initialise model using the tensorflow recommended layers

    grads = []
    num_of_vars = 0
    node_id = None

    def __init__(self, nid):
        super(Model_Node, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        self.num_of_vars = -1
        self.node_id = nid
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)


    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    def __getitem__(self, item): return self
    # initialising loss computatuin function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)

    # initialising stochastic gradient descent function
    # optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    # setting up the metrics collection tensors
    train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy'+str(node_id))

    test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='test_accuracy'+str(node_id))

    # model train function
    @tf.function
    def train_step(self, images, labels, glo=False, g=None):
        # if glo=true then it updates the global model using the gradient
        # given in argument g
        if glo:
            # self.optimizer.apply_gradients(zip(g, self.trainable_variables),experimental_aggregate_gradients=True)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            return
        with tf.GradientTape() as tape:
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables);
        if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
        # applies training gradients using stochastic gradient descend
        # self.optimizer.apply_gradients(zip(gradients,self.trainable_variables), experimental_aggregate_gradients=False)
        self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))
        # calculates loss and accuracy
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)
        x = self.num_of_vars
        self.grads = gradients[-x:]
        # returns the gradient
        return self.grads

    # model predict function
    @tf.function
    def test_step(self, images, labels):
        # predicts the label
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        # calculates loss and accuracy
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

# *****************************************************************************
# function for averaging the gradients
def avgs(matrices = []):
    segmentIDs = [0] * len(matrices)
    return tf.math.segment_mean(matrices, segmentIDs)

# function returning the enegry cost
def compute_transmission_cost(node):
    cost = 0
    for c in paths[node]: cost += (costs[int(c)] * ENERGY)
    return cost
# *****************************************************************************
start_time =  datetime.now().strftime("%H:%M:%S")

# sim run
mnist = tf.keras.datasets.mnist
print("loading dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(5)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

init = None
nds = [[], [], [], [], []]
i = 0
for img, lbl in train_ds:
        if i == 0: init = (img, lbl); i += 1; continue
        nds[i%5].append((img, lbl))
        i += 1
print("dataset split")
# *****************************************************************************
print("initialiasing nodes")
nodes = []
for i in range(0, NODES):
    nodes.append(Model_Node(i)); print("node", i, "initialised")
    nodes[-1].train_loss.reset_state()
    nodes[-1].train_accuracy.reset_state()
    nodes[-1].test_loss.reset_state()
    nodes[-1].test_accuracy.reset_state()
# *****************************************************************************
j, ecost = 0, 0
ecosts, acc, los, bits = [], [], [], []
gs = []
# initial train of all nodes
for i in range(0, NODES):
    nodes[i].train_step(init[0], init[1])
    if i == 0: continue
    nodes[i].set_weights(nodes[0].get_weights())
gsizes = []
totg = 0 #total number of gradients in model
for g in nodes[0].get_weights(): gsizes.append(g.size); totg += g.size
maxg = int(T_PERCENTAGE * totg) # number of max values to send
# main training and testing loop
for i in range(0, len(nds[0])):
    print("Iteration: ", i)
    # train the local models
    for j in range(1, NODES):
        print("training local model ", j)
        g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
    idiv = i % SGD_STEPS # checks how many steps of SGD have been run
    if idiv == (SGD_STEPS - 1):
        ws = nodes[0].get_weights()
        nd = [0] * (NODES - 1)
        # t-compress gradient (client side)
        for j in range(1, NODES):
            print("Starting gradient compression for node", j, "at",
                  datetime.now().strftime("%H:%M:%S"))
            g = nodes[j].get_weights()
            totsize, size = 0, 0
            for k in range(0, 6): g[k] = ws[k] - g[k]
            gflat = np.concatenate(g, axis=None)
            totsize = gflat.nbytes
            inds = np.argpartition(abs(gflat), -maxg)[-maxg:]
            size = np.array(inds).nbytes + np.array(gflat[inds]).nbytes
            gs.append([inds, gflat[inds]])
            percentage = (size / totsize) * 100
            nd[j-1] = size
            print("Node ", j, ": compressed and sent ",
                  percentage, "% of original gradient")
            print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
        # average differences (server side)
        for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
        d = {}
        for e in gs:
            for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
        for gi, gv in d.items():
            for k in range(0, 6):
                if gi < gsizes[k]:
                    ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
                    break
                gi -= gsizes[k]
        # update global model
        # nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables), experimental_aggregate_gradients=True)
        nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))

        # gather metrics for graphs
        for ti, tl in test_ds: nodes[0].test_step(ti, tl)
        acc.append(nodes[0].test_accuracy.result() * 100)
        los.append(nodes[0].test_loss.result())
        print("Accuracy: ", acc[-1])
        print("Loss: ", los[-1])
        for j in range(1, 6):
            ct = compute_transmission_cost(j)
            ecost += ct * nd[j-1] * 8
        ecosts.append(ecost)
        print("Energy cost: ", ecost)
        size = 0
        for n in nd: size += n * 8
        bits.append(size)
        print("Bits: ", bits[-1])
        gs = []
        for j in range(1, NODES):
            nodes[j].set_weights(nodes[0].get_weights())
        nodes[0].test_loss.reset_state()
        nodes[0].test_accuracy.reset_state()
# run another last iteration to account for the leftover records
print("Iteration: ", i+1)
print("Remainder: ", i % SGD_STEPS)
ws = nodes[0].get_weights()
nd = [0] * (NODES - 1)
# t-compress gradient (client side)
for j in range(1, NODES):
    print("Starting gradient compression for node", j, "at",
          datetime.now().strftime("%H:%M:%S"))
    g = nodes[j].get_weights()
    totsize, size = 0, 0
    for k in range(0, 6): g[k] = ws[k] - g[k]
    gflat = np.concatenate(g, axis=None)
    totsize = gflat.nbytes
    inds = np.argpartition(abs(gflat), -maxg)[-maxg:]
    size = np.array(inds).nbytes + np.array(gflat[inds]).nbytes
    gs.append([inds, gflat[inds]])
    percentage = (size / totsize) * 100
    nd[j-1] = size
    print("Node ", j, ": compressed and sent ",
          percentage, "% of original gradient")
    print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
# average differences (server side)
for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
d = {}
for e in gs:
    for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
for gi, gv in d.items():
    for k in range(0, 6):
        if gi < gsizes[k]:
            ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
            break
        gi -= gsizes[k]
# update global model
# nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
#                                    experimental_aggregate_gradients=True)
nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))

# gather metrics for graphs
for ti, tl in test_ds: nodes[0].test_step(ti, tl)
acc.append(nodes[0].test_accuracy.result() * 100)
los.append(nodes[0].test_loss.result())
print("Accuracy: ", acc[-1])
print("Loss: ", los[-1])
for j in range(1, 6):
    ct = compute_transmission_cost(j)
    ecost += ct * nd[j-1] * 8
ecosts.append(ecost)
print("Energy cost: ", ecost)
size = 0
for n in nd: size += n * 8
bits.append(size)
print("Bits: ", bits[-1])
# *****************************************************************************
# plot and save graphs to file
print("plotting figures")
plt.figure(1, figsize=(20,20))
plt.title('FL Accuracy', fontsize=25)
plt.plot(range(0, len(acc)), acc, alpha=0.8)
pngname = "baseline_acc_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(2, figsize=(20,20))
plt.title('FL size', fontsize=25)
plt.plot(range(0, len(bits)), bits, alpha=0.8)
pngname = "baseline_size_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(3, figsize=(20,20))
plt.title('FL Energy Consupmtioon', fontsize=25)
plt.plot(range(0, len(ecosts)), ecosts, alpha=0.8)
pngname = "baseline_energy_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')
# *****************************************************************************
print("script finished running")
print("start time: ", start_time, " --------- end time:",
      datetime.now().strftime("%H:%M:%S"))

loading dataset...
dataset split
initialiasing nodes
node 0 initialised
node 1 initialised
node 2 initialised
node 3 initialised
node 4 initialised
node 5 initialised


Streaming output truncated to the last 5000 lines.
Node  5 : compressed and sent  11.999961019752158 % of original gradient
Compression finished at 13:51:45
Accuracy:  tf.Tensor(77.880005, shape=(), dtype=float32)
Loss:  tf.Tensor(1.093998, shape=(), dtype=float32)
Energy cost:  8851814400.0
Bits:  53196000
Iteration:  160
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  161
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  162
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  163
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  164
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Starting gr

KeyboardInterrupt: 

In [12]:
####################   FL-CAT   ##################
from datetime import datetime
import tensorflow as tf
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import sys
import random as rand

# *****************************************************************************

# constant variables
ALPHA = 0.1 # learning rate
ENERGY = 2 # energy units cost per bit
NODES = 6 # num of nodes in network including sink
SGD_STEPS = 5 # num of stochastic gradient descent steps in between updates
T_PERCENTAGE = 0.04
# *****************************************************************************
# array of nodes in network
paths = {3 : "310", 4: "420", 5: "520", 1 : "10", 2 : "20"}
costs = {0: 0, 1: 3, 2: 1, 3: 0.5, 4: 2 , 5:1.5}
# *****************************************************************************
# setup of the neural network model's layers initialisation
class Model_Node(Model):
    # initialise model using the tensorflow recommended layers

    grads = []
    num_of_vars = 0
    node_id = None

    def __init__(self, nid):
        super(Model_Node, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        self.num_of_vars = -1
        self.node_id = nid
        # initialising stochastic gradient descent function
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    def __getitem__(self, item): return self
    # initialising loss computatuin function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)



    # setting up the metrics collection tensors
    train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy'+str(node_id))

    test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='test_accuracy'+str(node_id))

    # model train function
    @tf.function
    def train_step(self, images, labels, glo=False, g=None):
        # if glo=true then it updates the global model using the gradient
        # given in argument g
        if glo:
            # self.optimizer.apply_gradients(zip(g, self.trainable_variables),experimental_aggregate_gradients=True)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            return
        with tf.GradientTape() as tape:
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables);
        if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
        # applies training gradients using stochastic gradient descend
        # self.optimizer.apply_gradients(zip(gradients,
        #                                    self.trainable_variables),
        #                                experimental_aggregate_gradients=False)
        self.optimizer.apply_gradients(zip(gradients,
                                           self.trainable_variables))
        # calculates loss and accuracy
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)
        x = self.num_of_vars
        self.grads = gradients[-x:]
        # returns the gradient
        return self.grads

    # model predict function
    @tf.function
    def test_step(self, images, labels):
        # predicts the label
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        # calculates loss and accuracy
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

# *****************************************************************************
# function for averaging the gradients
def avgs(matrices = []):
    segmentIDs = [0] * len(matrices)
    return tf.math.segment_mean(matrices, segmentIDs)

# function returning the enegry cost
def compute_transmission_cost(node):
    cost = 0
    for c in paths[node]: cost += (costs[int(c)] * ENERGY)
    return cost
# *****************************************************************************
start_time =  datetime.now().strftime("%H:%M:%S")

# sim run
mnist = tf.keras.datasets.mnist
print("loading dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(1)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

init = None
nds = [[], [], [], [], []]
i = 0
for img, lbl in train_ds:
        if i == 0: init = (img, lbl); i += 1
        for j in range(0, 5):
            nds[j].append((img, lbl))
            nds[j].append((img, lbl))
            nds[j].append((img, lbl))
for i in range(0, 5): rand.shuffle(nds[i])
# print(len(nds[0])); sys.exit(0)
print("dataset split")
# *****************************************************************************
print("initialiasing nodes")
nodes = []
for i in range(0, NODES):
    nodes.append(Model_Node(i)); print("node", i, "initialised")
    nodes[-1].train_loss.reset_state()
    nodes[-1].train_accuracy.reset_state()
    nodes[-1].test_loss.reset_state()
    nodes[-1].test_accuracy.reset_state()
# *****************************************************************************
j, ecost = 0, 0
ecosts, acc, los, bits = [], [], [], []
gs = []
# initial train of all nodes
for i in range(0, NODES):
    nodes[i].train_step(init[0], init[1])
    if i == 0: continue
    nodes[i].set_weights(nodes[0].get_weights())
gsizes = []
totg = 0 #total number of gradients in model
for g in nodes[0].get_weights(): gsizes.append(g.size); totg += g.size
maxg = int(T_PERCENTAGE * totg) # number of max values to send
# main training and testing loop
for i in range(0, len(nds[0])):
    print("Iteration: ", i)
    # train the local models
    for j in range(1, NODES):
        print("training local model ", j)
        g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
    idiv = i % SGD_STEPS # checks how many steps of SGD have been run
    if idiv == (SGD_STEPS - 1):
        ws = nodes[0].get_weights()
        nd = [0] * (NODES - 1)
        # t-compress gradient (client side)
        for j in range(1, NODES):
            print("Starting gradient compression for node", j, "at",
                  datetime.now().strftime("%H:%M:%S"))
            g = nodes[j].get_weights()
            totsize, size = 0, 0
            for k in range(0, 6): g[k] = ws[k] - g[k]
            gflat = np.concatenate(g, axis=None)
            totsize = gflat.nbytes
            inds = np.argsort(abs(gflat))
            cc = compute_transmission_cost(j) * 8
            minval, minind = 0, 0
            for k in range(1, len(inds)+1):
                compressed = gflat[inds[-k:]]
                calc = (la.norm(compressed)**2) / (la.norm(gflat)**2)
                calc /= (cc * compressed.size)
                if calc > minval: minval = calc; minind = k; continue
                if calc < minval: break
            size = np.array(inds[-minind:]).nbytes + np.array(
                gflat[inds[-minind:]]).nbytes
            gs.append([inds[-minind:], gflat[inds[-minind:]]])
            percentage = (size / totsize) * 100
            nd[j-1] = size
            print("Node ", j, ": compressed and sent ",
                  percentage, "% of original gradient using", k-1, "gradients")
            print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
        # average differences (server side)
        for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
        d = {}
        for e in gs:
            for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
        for gi, gv in d.items():
            for k in range(0, 6):
                if gi < gsizes[k]:
                    ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
                    break
                gi -= gsizes[k]
        # update global model
        # nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
        #                                    experimental_aggregate_gradients=True)
        nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))
        # gather metrics for graphs
        for ti, tl in test_ds: nodes[0].test_step(ti, tl)
        acc.append(nodes[0].test_accuracy.result() * 100)
        los.append(nodes[0].test_loss.result())
        print("Accuracy: ", acc[-1])
        print("Loss:", los[-1])
        for j in range(1, 6):
            ct = compute_transmission_cost(j)
            ecost += ct * nd[j-1] * 8
        ecosts.append(ecost)
        print("Energy cost: ", ecost)
        size = 0
        for n in nd: size += n * 8
        bits.append(size)
        print("Bits: ", bits[-1])
        gs = []
        for j in range(1, NODES): nodes[j].set_weights(nodes[0].get_weights())
        nodes[0].test_loss.reset_state()
        nodes[0].test_accuracy.reset_state()
"""# run another last iteration to account for the leftover records
print("Iteration: ", i+1)
print("Remainder: ", i % SGD_STEPS)
ws = nodes[0].get_weights()
nd = [0] * (NODES - 1)
# t-compress gradient (client side)
for j in range(1, NODES):
    print("Starting gradient compression for node", j, "at",
          datetime.now().strftime("%H:%M:%S"))
    g = nodes[j].get_weights()
    totsize, size = 0, 0
    for k in range(0, 6): g[k] = ws[k] - g[k]
    gflat = np.concatenate(g, axis=None)
    totsize = gflat.nbytes
    inds = np.argsort(abs(gflat))
    cc = compute_transmission_cost(j) * 8
    minval, minind = 0, 0
    for k in range(1, len(inds)+1):
        compressed = gflat[inds[-k:]]
        calc = (la.norm(compressed)**2) / (la.norm(gflat)**2)
        calc /= (cc * compressed.size)
        if calc > minval: minval = calc; minind = k; continue
        if calc < minval: break
    size = np.array(inds[-minind:]).nbytes + np.array(
        gflat[inds[-minind:]]).nbytes
    gs.append([inds[-minind:], gflat[inds[-minind:]]])
    percentage = (size / totsize) * 100
    nd[j-1] = size
    print("Node ", j, ": compressed and sent ",
          percentage, "% of original gradient using", k, "gradients")
    print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
# average differences (server side)
for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
d = {}
for e in gs:
    for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
for gi, gv in d.items():
    for k in range(0, 6):
        if gi < gsizes[k]:
            ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
            break
        gi -= gsizes[k]
# update global model
nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
                                   experimental_aggregate_gradients=True)
"""
# gather metrics for graphs
for ti, tl in test_ds: nodes[0].test_step(ti, tl)
acc.append(nodes[0].test_accuracy.result() * 100)
los.append(nodes[0].test_loss.result())
print("Accuracy: ", acc[-1])
print("Loss:", los[-1])
for j in range(1, 6):
    ct = compute_transmission_cost(j)
    ecost += ct * nd[j-1] * 8
ecosts.append(ecost)
print("Energy cost: ", ecost)
size = 0
for n in nd: size += n * 8
bits.append(size)
print("Bits: ", bits[-1])
# *****************************************************************************
# plot and save graphs to file
print("plotting figures")
plt.figure(1, figsize=(20,20))
plt.title('FL Accuracy', fontsize=25)
plt.plot(range(0, len(acc)), acc, alpha=0.8)
pngname = "baseline_acc_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(2, figsize=(20,20))
plt.title('FL size', fontsize=25)
plt.plot(range(0, len(bits)), bits, alpha=0.8)
pngname = "baseline_size_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(3, figsize=(20,20))
plt.title('FL Energy Consupmtioon', fontsize=25)
plt.plot(range(0, len(ecosts)), ecosts, alpha=0.8)
pngname = "baseline_energy_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')
# *****************************************************************************
print("script finished running")
print("start time: ", start_time, " --------- end time:",
      datetime.now().strftime("%H:%M:%S"))

Streaming output truncated to the last 5000 lines.
Compression finished at 15:23:46
Starting gradient compression for node 2 at 15:23:46
Node  2 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 15:23:46
Starting gradient compression for node 3 at 15:23:46
Node  3 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 15:23:46
Starting gradient compression for node 4 at 15:23:46
Node  4 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 15:23:46
Starting gradient compression for node 5 at 15:23:46
Node  5 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 15:23:46
Accuracy:  tf.Tensor(14.830001, shape=(), dtype=float32)
Loss: tf.Tensor(2.2904997, shape=(), dtype=float32)
Energy cost:  1799616.0
Bits:  480
Iteration:  3605
training local model  1
trainin

KeyboardInterrupt: 

In [13]:
############  CAT SPARSIFICATION   #############
from datetime import datetime
import tensorflow as tf
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import sys
import random as rand

# *****************************************************************************

# constant variables
ALPHA = 0.1 # learning rate
ENERGY = 2 # energy units cost per bit
NODES = 6 # num of nodes in network including sink
SGD_STEPS = 5 # num of stochastic gradient descent steps in between updates
T_PERCENTAGE = 0.04
# *****************************************************************************
# array of nodes in network
paths = {3 : "310", 4: "420", 5: "520", 1 : "10", 2 : "20"}
costs = {0: 0, 1: 3, 2: 1, 3: 0.5, 4: 2 , 5:1.5}
# *****************************************************************************
# setup of the neural network model's layers initialisation
class Model_Node(Model):
    # initialise model using the tensorflow recommended layers

    grads = []
    num_of_vars = 0
    node_id = None

    def __init__(self, nid):
        super(Model_Node, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
        self.num_of_vars = -1
        self.node_id = nid
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    def __getitem__(self, item): return self
    # initialising loss computatuin function
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True)

    # initialising stochastic gradient descent function
    # optimizer = tf.keras.optimizers.SGD(learning_rate=ALPHA)

    # setting up the metrics collection tensors
    train_loss = tf.keras.metrics.Mean(name='train_loss'+str(node_id))
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='train_accuracy'+str(node_id))

    test_loss = tf.keras.metrics.Mean(name='test_loss'+str(node_id))
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
        name='test_accuracy'+str(node_id))

    # model train function
    @tf.function
    def train_step(self, images, labels, glo=False, g=None):
        # if glo=true then it updates the global model using the gradient
        # given in argument g
        if glo:
            # self.optimizer.apply_gradients(zip(g, self.trainable_variables),
            #                                experimental_aggregate_gradients=True)
            self.optimizer.apply_gradients(zip(g, self.trainable_variables))
            return
        with tf.GradientTape() as tape:
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        gradients = tape.gradient(loss, self.trainable_variables);
        if self.num_of_vars < 0: self.num_of_vars = len(self.trainable_variables)
        # applies training gradients using stochastic gradient descend
        # self.optimizer.apply_gradients(zip(gradients,
        #                                    self.trainable_variables),
        #                                experimental_aggregate_gradients=False)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # calculates loss and accuracy
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)
        x = self.num_of_vars
        self.grads = gradients[-x:]
        # returns the gradient
        return self.grads

    # model predict function
    @tf.function
    def test_step(self, images, labels):
        # predicts the label
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        # calculates loss and accuracy
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)

# *****************************************************************************
# function for averaging the gradients
def avgs(matrices = []):
    segmentIDs = [0] * len(matrices)
    return tf.math.segment_mean(matrices, segmentIDs)

# function returning the enegry cost
def compute_transmission_cost(node):
    cost = 0
    for c in paths[node]: cost += (costs[int(c)] * ENERGY)
    return cost
# *****************************************************************************
start_time =  datetime.now().strftime("%H:%M:%S")

# sim run
mnist = tf.keras.datasets.mnist
print("loading dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(1)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

init = None
nds = [[], [], [], [], []]
i = 0
for img, lbl in train_ds:
        if i == 0: init = (img, lbl); i += 1
        for j in range(0, 5):
            nds[j].append((img, lbl))
            nds[j].append((img, lbl))
            nds[j].append((img, lbl))
for i in range(0, 5): rand.shuffle(nds[i])
# print(len(nds[0])); sys.exit(0)
print("dataset split")
# *****************************************************************************
print("initialiasing nodes")
nodes = []
for i in range(0, NODES):
    nodes.append(Model_Node(i)); print("node", i, "initialised")
    nodes[-1].train_loss.reset_state()
    nodes[-1].train_accuracy.reset_state()
    nodes[-1].test_loss.reset_state()
    nodes[-1].test_accuracy.reset_state()
# *****************************************************************************
j, ecost = 0, 0
ecosts, acc, los, bits = [], [], [], []
gs = []
# initial train of all nodes
for i in range(0, NODES):
    nodes[i].train_step(init[0], init[1])
    if i == 0: continue
    nodes[i].set_weights(nodes[0].get_weights())
gsizes = []
totg = 0 #total number of gradients in model
for g in nodes[0].get_weights(): gsizes.append(g.size); totg += g.size
maxg = int(T_PERCENTAGE * totg) # number of max values to send
# main training and testing loop
for i in range(0, len(nds[0])):
    print("Iteration: ", i)
    # train the local models
    for j in range(1, NODES):
        print("training local model ", j)
        g = nodes[j].train_step(nds[j-1][i][0], nds[j-1][i][1])
    idiv = i % SGD_STEPS # checks how many steps of SGD have been run
    if idiv == (SGD_STEPS - 1):
        ws = nodes[0].get_weights()
        nd = [0] * (NODES - 1)
        # t-compress gradient (client side)
        for j in range(1, NODES):
            print("Starting gradient compression for node", j, "at",
                  datetime.now().strftime("%H:%M:%S"))
            g = nodes[j].get_weights()
            totsize, size = 0, 0
            for k in range(0, 6): g[k] = ws[k] - g[k]
            gflat = np.concatenate(g, axis=None)
            totsize = gflat.nbytes
            inds = np.argsort(abs(gflat))
            cc = compute_transmission_cost(j) * 8
            minval, minind = 0, 0
            for k in range(1, len(inds)+1):
                compressed = gflat[inds[-k:]]
                calc = (la.norm(compressed)**2) / (la.norm(gflat)**2)
                calc /= (cc * compressed.size)
                if calc > minval: minval = calc; minind = k; continue
                if calc < minval: break
            size = np.array(inds[-minind:]).nbytes + np.array(
                gflat[inds[-minind:]]).nbytes
            gs.append([inds[-minind:], gflat[inds[-minind:]]])
            percentage = (size / totsize) * 100
            nd[j-1] = size
            print("Node ", j, ": compressed and sent ",
                  percentage, "% of original gradient using", k-1, "gradients")
            print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
        # average differences (server side)
        for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
        d = {}
        for e in gs:
            for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
        for gi, gv in d.items():
            for k in range(0, 6):
                if gi < gsizes[k]:
                    ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
                    break
                gi -= gsizes[k]
        # update global model
        # nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
        #                                    experimental_aggregate_gradients=True)
        nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables))
        # gather metrics for graphs
        for ti, tl in test_ds: nodes[0].test_step(ti, tl)
        acc.append(nodes[0].test_accuracy.result() * 100)
        los.append(nodes[0].test_loss.result())
        print("Accuracy: ", acc[-1])
        print("Loss:", los[-1])
        for j in range(1, 6):
            ct = compute_transmission_cost(j)
            ecost += ct * nd[j-1] * 8
        ecosts.append(ecost)
        print("Energy cost: ", ecost)
        size = 0
        for n in nd: size += n * 8
        bits.append(size)
        print("Bits: ", bits[-1])
        gs = []
        for j in range(1, NODES): nodes[j].set_weights(nodes[0].get_weights())
        nodes[0].test_loss.reset_state()
        nodes[0].test_accuracy.reset_state()
"""# run another last iteration to account for the leftover records
print("Iteration: ", i+1)
print("Remainder: ", i % SGD_STEPS)
ws = nodes[0].get_weights()
nd = [0] * (NODES - 1)
# t-compress gradient (client side)
for j in range(1, NODES):
    print("Starting gradient compression for node", j, "at",
          datetime.now().strftime("%H:%M:%S"))
    g = nodes[j].get_weights()
    totsize, size = 0, 0
    for k in range(0, 6): g[k] = ws[k] - g[k]
    gflat = np.concatenate(g, axis=None)
    totsize = gflat.nbytes
    inds = np.argsort(abs(gflat))
    cc = compute_transmission_cost(j) * 8
    minval, minind = 0, 0
    for k in range(1, len(inds)+1):
        compressed = gflat[inds[-k:]]
        calc = (la.norm(compressed)**2) / (la.norm(gflat)**2)
        calc /= (cc * compressed.size)
        if calc > minval: minval = calc; minind = k; continue
        if calc < minval: break
    size = np.array(inds[-minind:]).nbytes + np.array(
        gflat[inds[-minind:]]).nbytes
    gs.append([inds[-minind:], gflat[inds[-minind:]]])
    percentage = (size / totsize) * 100
    nd[j-1] = size
    print("Node ", j, ": compressed and sent ",
          percentage, "% of original gradient using", k, "gradients")
    print("Compression finished at", datetime.now().strftime("%H:%M:%S"))
# average differences (server side)
for j in range(0, 6): ws[j] = np.zeros(ws[j].shape, dtype="float32")
d = {}
for e in gs:
    for gi, gv in zip(e[0], e[1]): d.setdefault(gi, []).append(gv)
for gi, gv in d.items():
    for k in range(0, 6):
        if gi < gsizes[k]:
            ws[k][np.unravel_index(gi, ws[k].shape)] = np.mean(gv)
            break
        gi -= gsizes[k]
# update global model
nodes[0].optimizer.apply_gradients(zip(ws,nodes[0].trainable_variables),
                                   experimental_aggregate_gradients=True)
"""
# gather metrics for graphs
for ti, tl in test_ds: nodes[0].test_step(ti, tl)
acc.append(nodes[0].test_accuracy.result() * 100)
los.append(nodes[0].test_loss.result())
print("Accuracy: ", acc[-1])
print("Loss:", los[-1])
for j in range(1, 6):
    ct = compute_transmission_cost(j)
    ecost += ct * nd[j-1] * 8
ecosts.append(ecost)
print("Energy cost: ", ecost)
size = 0
for n in nd: size += n * 8
bits.append(size)
print("Bits: ", bits[-1])
# *****************************************************************************
# plot and save graphs to file
print("plotting figures")
plt.figure(1, figsize=(20,20))
plt.title('FL Accuracy', fontsize=25)
plt.plot(range(0, len(acc)), acc, alpha=0.8)
pngname = "baseline_acc_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(2, figsize=(20,20))
plt.title('FL size', fontsize=25)
plt.plot(range(0, len(bits)), bits, alpha=0.8)
pngname = "baseline_size_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')

plt.figure(3, figsize=(20,20))
plt.title('FL Energy Consupmtioon', fontsize=25)
plt.plot(range(0, len(ecosts)), ecosts, alpha=0.8)
pngname = "baseline_energy_tcompressed.png"
print(pngname)
plt.savefig(pngname, bbox_inches='tight')
# *****************************************************************************
print("script finished running")
print("start time: ", start_time, " --------- end time:",
      datetime.now().strftime("%H:%M:%S"))

Streaming output truncated to the last 5000 lines.
training local model  3
training local model  4
training local model  5
Iteration:  4408
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Iteration:  4409
training local model  1
training local model  2
training local model  3
training local model  4
training local model  5
Starting gradient compression for node 1 at 16:35:03
Node  1 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 16:35:03
Starting gradient compression for node 2 at 16:35:03
Node  2 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 16:35:03
Starting gradient compression for node 3 at 16:35:03
Node  3 : compressed and sent  0.00010827846622830733 % of original gradient using 1 gradients
Compression finished at 16:35:03
Starting gradient compression for node 4 at 16:35:03
Node  4

KeyboardInterrupt: 